In [1]:
from qc.word_generator import WordGenerator
from qc.bloch_matrix import *
from qutip import Bloch
from picos import RealVariable, Problem
import picos as pc

In [2]:
rn0 = np.random.default_rng().normal(size=3)
n0 = rn0 /np.linalg.norm(rn0)

output_file = open("output/out.txt", "a")

In [4]:
for length in range(4,15):
    wg = WordGenerator(['H','T'], length).generate_words_shorter_than()
    m = get_bloch_matrices(wg, 0.95)
    v = get_bloch_vectors(m, [1,0,0])

    problem = Problem()
    N = len(v)
    p = {}

    # dodaję zmienne
    for i in range(N - 1):
        p[i] = problem.add_variable('p[{0}]'.format(i))
    t = problem.add_variable('t')

    # każde p >= 0
    problem.add_list_of_constraints([p[i] >= 0 for i in range(N - 1)])
    # p sumują się do 1
    problem.add_constraint(1 == pc.sum([p[i] for i in range(N - 1)]))
    # wiąz na wektory
    problem.add_constraint(t * n0[0] == pc.sum([p[j] * v[j][0] for j in range(N - 1)]))
    problem.add_constraint(t * n0[1] == pc.sum([p[j] * v[j][1] for j in range(N - 1)]))
    problem.add_constraint(t * n0[2] == pc.sum([p[j] * v[j][2] for j in range(N - 1)]))  

    problem.set_objective("max", t)
    solution = problem.solve(solver='mosek')
    output_file.write(str(t) + "\t" + str(length) + "\n")
output_file.close()

<ipython-input-4-c408c3b9ae1c>:12: DeprecationWarning: Problem.add_variable is deprecated: Variables can now be created independent of problems, and do not need to be added to any problem explicitly.
  p[i] = problem.add_variable('p[{0}]'.format(i))
<ipython-input-4-c408c3b9ae1c>:13: DeprecationWarning: Problem.add_variable is deprecated: Variables can now be created independent of problems, and do not need to be added to any problem explicitly.
  t = problem.add_variable('t')


KeyboardInterrupt: 

In [ ]:
b = Bloch()
vx, vy, vz = np.transpose(v)
b.add_points([vx, vy, vz])
b.render()